# Types and Multiple Dispatch

Pearl Li

December 18, 2017

## Outline

1. Types
2. Why Use Types?
3. Multiple Dispatch
4. Writing Julian Code

## Types

A **data type** is a classification identifying the kind of data you have. An object’s type determines the possible values it can take on, which operations and functions can be applied to it, and how the computer stores it.

Examples:

- Numeric types: `Int64`, `Float64`
- String types: `String`, `SubString`
- `Bool`
- `Array`

Names of types are written in UpperCamelCase.

A **concrete instance** (also an object or a value) of a type `T` is a piece of data in memory that has type `T`.

Variables are not data, but are simply names that point/refer to a specific piece of data. The underlying data that a variable refers to has a specific type.

In [1]:
# What is the type of 10?
typeof(10)

Int64

In [2]:
# Is 10 an Int64?
isa(10, Int64)

true

In [3]:
# What is the type of the elements of an array?
X = [1.0, 2.0, 3.0]
eltype(X)

Float64

### Composite Types

A **composite type** is a collection of named fields that can be treated as a single value. They bear a passing resemblance to MATLAB structs.

All fields must be declared ahead of time. The double colon, `::`, constrains a field to contain values of a certain type. This is optional for any field.

**Mutable** composite types are defined using the keywords `mutable struct`.

In [4]:
# Type definition
mutable struct Parameter
    value::Float64
    transformation::Function # Function is a type!
    tex_label::String
    description::String
end

When a type with $n$ fields is defined, a constructor (function that creates an instance of that type) that takes $n$ ordered arguments is automatically created.

In [5]:
# Creating an instance of the Parameter type using the default
# constructor
β = Parameter(0.9, identity, "\beta", "Discount rate")

Parameter(0.9, identity, "\beta", "Discount rate")

Additional constructors can be defined for convenience.

In [6]:
# Alternative constructors end with an appeal to the default
# constructor
function Parameter(value::Float64, tex_label::String)
    transformation = identity
    description = "No description available"
    return Parameter(value, transformation, tex_label, description)
end

α = Parameter(0.5, "\alpha")

Parameter(0.5, identity, "\alpha", "No description available")

In [10]:
# Find the fields of an instance of a composite type
fieldnames(α)

4-element Array{Symbol,1}:
 :value         
 :transformation
 :tex_label     
 :description   

In [11]:
# Access a particular field using .
α.value

0.5

In [12]:
# Fields in a mutable composite type are modifiable and can be assigned to, like 
# ordinary variables
α.value = 0.75

0.75

### Subtyping

Types are hierarchically related to each other. All are subtypes of the `Any` type.

There are two main kinds of types in Julia:

1. Concrete types: familiar types that you can create instances of, like `Int64` or `Float64`.
2. Abstract types: nodes in a type graph that serve to group similar kinds of objects. Abstract types cannot be instantiated and do not have explicitly declared fields. For example, `Integer` or `Number`.

In [5]:
# Define an abstract type
abstract type Model end

In [10]:
# Define concrete subtypes of that abstract type
mutable struct VAR <: Model
    n_lags::Int64
    variables::Vector{Symbol} # Ny x 1
    data::Matrix{Float64}     # Ny x Nt
end

In [15]:
# Check subtyping relation
VAR <: Model

true

In [16]:
# Instantiate a VAR
data = [3.2 3.5 2.9 3.0; # GDP series
        1.5 1.2 1.4 1.9] # inflation series
model = VAR(4, [:gdp, :inflation], data)

VAR(4, Symbol[:gdp, :inflation], [3.2 3.5 2.9 3.0; 1.5 1.2 1.4 1.9])

In [17]:
# Instances of the VAR type are also instances of the Model type
isa(model, Model)

true

In [18]:
# Why does this throw an error?
3 <: Number

LoadError: [91mTypeError: issubtype: expected Type, got Int64[39m

### Parameterized Types

**Parameterized types** are data types that are defined to handle values identically regardless of the type of those values.

Arrays are a familiar example. An `Array{T,1}` is a one-dimensional array filled with objects of any type `T` (e.g. `Float64`, `String`).

In [25]:
# Defining a parametric point
struct Duple{T} # T is a parameter to the type Duple
    x::T
    y::T
end

This single declaration defines an unlimited number of new types: `Duple{String}`, `Duple{Float64}`, etc. are all immediately usable.

In [20]:
Duple(3, -15)

Duple{Int64}(3, -15)

In [21]:
Duple("Broadway", "42nd St")

Duple{String}("Broadway", "42nd St")

In [22]:
# What happens here?
Duple(1.5, 3)

LoadError: [91mMethodError: no method matching Duple(::Float64, ::Int64)[0m
Closest candidates are:
  Duple(::T, [91m::T[39m) where T at In[19]:3[39m

Notice that we defined `Duple` without the `mutable` keyword. We can still access the fields of `Duple` instances as before, but now attempts to modify the fields will cause an error to be thrown.

In [27]:
pt = Duple(3, -15)
pt.x

3

In [28]:
pt.x = 0

LoadError: [91mtype Duple is immutable[39m

Each particular parameterization of a parametric type is a type, and moreover a subtype of the parametric type.

In [29]:
typeof(Duple{Int})

DataType

In [30]:
Duple{Int} <: Duple

true

We can also restrict the type parameter `T`:

In [31]:
# T can be any subtype of Number, but nothing else
type PlanarCoordinate{T<:Number}
    x::T
    y::T
end

In [32]:
PlanarCoordinate("4th Ave", "14th St")

LoadError: [91mMethodError: no method matching PlanarCoordinate(::String, ::String)[39m

### Type Aliases

**Type aliases** can be used to simplify references to existing types using regular assignment. For example, `Vector{T}` and `Matrix{T}` are type aliases for `Array{T, 1}` and `Array{T, 2}` respectively:

In [33]:
Vector{T} = Array{T, 1}
Matrix{T} = Array{T, 2}

Array{T,2} where T

In [34]:
x = [1, 2, 3]
isa(x, Array{Int, 1})

true

In [35]:
isa(x, Vector{Int})

true

In [36]:
Vector{Int} == Array{Int, 1}

true

## Exercise

In [1]:
using Distributions

The [Distributions](https://juliastats.github.io/Distributions.jl/latest/index.html) package implements sampling, moments, PDFs, and more for many distributions. `Distribution` is an abstract parameterized type, and `ContinuousUnivariateDistribution` and `ContinuousMultivariateDistribution` are two specific (aliases for) abstract subtypes of `Distribution`.

```
abstract Distribution{F<:VariateForm,S<:ValueSupport}
const ContinuousUnivariateDistribution =
    Distribution{Univariate, Continuous}
const ContinuousMultivariateDistribution =
    Distribution{Multivariate, Continuous}
```

Implement the [triangular distribution](https://en.wikipedia.org/wiki/Triangular_distribution) ("a continuous probability distribution with lower limit $a$, upper limit $b$ and mode $c$, where $a < b$ and $a \leq c \leq b$") as an immutable composite type `MyTriangular`, which is a subtype of `ContinuousUnivariateDistribution`.

In [2]:
struct MyTriangular <: ContinuousUnivariateDistribution
    a::Real # lower limit
    b::Real # upper limit
    c::Real # mode

    # This "inner constructor" enforces type invariants
    function MyTriangular(a, b, c)
        @assert a < b
        @assert a <= c <= b
        return new(a, b, c)
    end
end

In addition to the default constructor, implement a second `Triangular` constructor which only takes in the lower and upper limits as arguments, setting the mode to $\frac{a+b}{2}$.

In [3]:
function MyTriangular(a, b)
    c = (a+b)/2
    return MyTriangular(a, b, c)
end

MyTriangular

## Why Use Types?

You can write all your code without thinking about types at all. If you do this, however, you’ll be missing out on some of the biggest benefits of using Julia.

If you understand types, you can:

- Write faster code
- Write expressive, clear, and well-structured programs (keep this in mind when we talk about functions)
- Reason more clearly about how your code works

From the QuantEcon lecture on [Types, Methods, and Dispatch](https://lectures.quantecon.org/jl/types_methods.html#motivation):

> At our respective homes we both have drawers full of fishing gear. Of course we have drawers full of other things too, like kitchen utensils, or clothes. Are these drawers really necessary? Perhaps not, but who wants to search the whole house for their fishing reel when the fish are biting? Certainly not us. Just as it’s convenient to store household objects in drawers, **it’s also convenient to organize the objects in your program into designated “containers”**.

Even if you only use built-in functions and types, your code still takes advantage of Julia’s type system. That’s why it’s important to understand what types are and how to use them.

In [29]:
# Example: writing type-stable functions
function sumofsins_unstable(n::Int)  
    sum = 0
    for i in 1:n
        sum += sin(3.4)
    end
    return sum
end  

function sumofsins_stable(n::Int)  
    sum = 0.0
    for i in 1:n
        sum += sin(3.4)
    end
    return sum
end

# Compile and run
sumofsins_unstable(1e5)
sumofsins_stable(1e5)

-25554.110202663698

In [30]:
@time sumofsins_unstable(1e5)

  0.014902 seconds (300.00 k allocations: 4.578 MiB, 62.10% gc time)


-25554.110202663698

In [31]:
@time sumofsins_stable(1e5)

  0.002460 seconds (5 allocations: 176 bytes)


-25554.110202663698

In `sumofsins_stable`, the compiler is guaranteed that `sum` is of type `Float64` throughout; therefore, it saves time and memory. On the other hand, in `sumofsins_unstable`, the compiler must check the type of `sum` at each iteration of the loop. Let's look at the LLVM [intermediate representation](http://www.johnmyleswhite.com/notebook/2013/12/06/writing-type-stable-code-in-julia/).

### Exercise

(Taken from QuantEcon's [Vectors, Arrays, and Matrices](https://lectures.quantecon.org/jl/julia_arrays.html) lecture.)

Write a function `solve_discrete_lyapunov` that solves the discrete Lyapunov equation $$S = ASA' + \Sigma \Sigma'$$ using the iterative procedure $$S_0 = \Sigma \Sigma'$$ $$S_{t+1} = A S_t A' + \Sigma \Sigma'$$ taking in as arguments the $n \times n$ matrix $A$, the $n \times k$ matrix $\Sigma$, and a number of iterations. 
You can assume that your $A$ and $\Sigma$ matrices are of type `Matrix{Float64}` and the number of iterations is an `Integer`. Make sure to check your code for type stability!

Solution: https://lectures.quantecon.org/jl/julia_arrays.html#solutions (Note that in the solutions, the function is called `compute_asymptotic_var`.)

In [ ]:
function solve_discrete_lyapunov(...)
    ...
end

## Multiple Dispatch

So far we have defined functions over argument lists of any type. Methods allow us to define functions “piecewise”. For any set of input arguments, we can define a **method**, a definition of one possible behavior for a function.

In [32]:
# Define one method of the function print_type
function print_type(x::Number)
    println("$x is a number")
end

print_type (generic function with 1 method)

In [33]:
# Define another method
function print_type(x::String)
    println("$x is a string")
end

print_type (generic function with 2 methods)

In [34]:
# Define yet another method
function print_type(x::Number, y::Number)
    println("$x and $y are both numbers")
end

print_type (generic function with 3 methods)

In [35]:
# See all methods for a given function
methods(print_type)

# 3 methods for generic function "print_type":
print_type(x::String) in Main at In[33]:3
print_type(x::Number) in Main at In[32]:3
print_type(x::Number, y::Number) in Main at In[34]:3

Julia uses **multiple dispatch** to decide which method of a function to execute when a function is applied. In particular, Julia compares the types of _all_ arguments to the signatures of the function’s methods in order to choose the applicable one, not just the first (hence "multiple").

In [36]:
print_type(5)

5 is a number


In [37]:
print_type("foo")

foo is a string


In [38]:
# This throws an error because no method of print_type has been
# defined for this set of arguments
print_type([1, 2, 3])

LoadError: [91mMethodError: no method matching print_type(::Array{Int64,1})[0m
Closest candidates are:
  print_type([91m::String[39m) at In[33]:3
  print_type([91m::Number[39m) at In[32]:3
  print_type([91m::Number[39m, [91m::Number[39m) at In[34]:3[39m

How is multiple dispatch useful for economic research? Recall that we defined the type `VAR` earlier, and made it a subtype of our abstract type `Model`. Let's define another subtype of `Model`:

In [6]:
# Define a linear regression model
type LinearRegression <: Model
    y_var::Symbol
    x_vars::Vector{Symbol}
    y_data::Vector{Float64} # N x 1
    x_data::Matrix{Float64} # N x Nx
end

Now we can use the same function name, `estimate_params`, to define different estimation behaviors for the different subtypes of `Model`:

In [11]:
function estimate_params(model::LinearRegression)
    # Estimate a linear regression using OLS
end

function estimate_params(model::VAR)
    # Estimate a VAR using maximum likelihood
end

function estimate_params(model::VAR, prior::Distribution)
    # Estimate a Bayesian VAR
end

estimate_params (generic function with 3 methods)

In [12]:
methods(estimate_params)

# 3 methods for generic function "estimate_params":
estimate_params(model::VAR) in Main at In[11]:7
estimate_params(model::LinearRegression) in Main at In[11]:3
estimate_params(model::VAR, prior::Distributions.Distribution) in Main at In[11]:11

### Exercise

Implement the function `estimate(model::LinearRegression)` using the given `LinearRegression` type. That is, return a $N_x \times 1$ vector of coefficients estimated using OLS. You may find the `inv` and `transpose` functions helpful. (As in MATLAB, you can also transpose matrices using `'`.)

In [14]:
function estimate_params(model::LinearRegression)
    X, Y = model.x_data, model.y_data
    β_hat = inv(X'*X) * (X'*Y)
    return vec(β_hat)
end

estimate_params (generic function with 3 methods)

Test it on the following model:

In [16]:
# Define dimensions
Nx = 2
N  = 1000

# Generate data
β = ones(Nx, 1)
X = rand(N, Nx)
ϵ = randn(N)
Y = X*β + ϵ

# Initialize and estimate model
model = LinearRegression(:y, [:x1, :x2], vec(Y), X)
β_hat = estimate_params(model)

2-element Array{Float64,1}:
 0.885933
 1.00956 

### Another Exercise

Recall the `Triangular` type we defined earlier:

In [20]:
struct Triangular <: ContinuousUnivariateDistribution
    a::Real # lower limit
    b::Real # upper limit
    c::Real # mode

    # This "inner constructor" enforces type invariants
    function Triangular(a, b, c)
        @assert a < b
        @assert a <= c <= b
        return new(a, b, c)
    end
end

We can also add additional methods to functions defined in outside packages. For example, we can extend `Distributions.cdf` to handle our `Triangular` type. Implement the triangular distribution CDF:

$$
f(x) = \begin{cases}
       0,                              &     x \leq a \\
       \frac{(x-a)^2}{(b-a)(c-a)},     & a < x \leq c \\
       1 - \frac{(b-x)^2}{(b-a)(b-c)}, & c < x <    b \\
       1,                              & \mathrm{otherwise}
       \end{cases}
$$

In [22]:
function Distributions.cdf(d::Triangular, x::Real)
    # Simplify notation
    a, b, c = d.a, d.b, d.c
    
    if x <= a
        return 0
    elseif a < x <= c
        return (x-a)^2 / ((b-a)*(c-a))
    elseif c < x < b
        return 1 - (b-x)^2 / ((b-a)*(b-c))
    else
        return 1
    end
end

Test your method below:

In [23]:
d = Triangular(0, 1, 0.5)
@assert cdf(d, 0)   == 0
@assert cdf(d, 0.5) == 0.5
@assert cdf(d, 1)   == 1

## Writing Julian Code

How is Julia so fast? Julia is just-in-time (JIT) compiled, which  means (according to [this StackExchange answer](http://stackoverflow.com/questions/95635/what-does-a-just-in-time-jit-compiler-do)):

> A JIT compiler runs after the program has started and compiles the code (usually bytecode or some kind of VM instructions) on the fly (or just-in-time, as it's called) into a form that's usually faster, typically the host CPU's native instruction set. **A JIT has access to dynamic runtime information whereas a standard compiler doesn't and can make better optimizations like inlining functions that are used frequently.**

> This is in contrast to a traditional compiler that compiles all the code to machine language before the program is first run.

In particular, Julia uses type information at runtime to optimize how your code is compiled. This is why writing type-stable code makes such a difference in speed!

As we've seen, you can use Julia just like you use MATLAB and get faster code. However, to write faster and _better_ code, attempt to write in a “Julian” manner:

- Define composite types as logically needed
- Write type-stable functions for best performance
- Take advantage of multiple dispatch to write code that looks like math
- Add methods to existing functions